In [1]:
# Import necessary libraries
from openprompt.plms import T5TokenizerWrapper
from datasets import load_from_disk
from openprompt.pipeline_base import PromptDataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer
from openprompt.prompts import PrefixTuningTemplate
from openprompt import PromptForClassification
from openprompt.data_utils import FewShotSampler
from openprompt.prompts import ManualVerbalizer
from openprompt.data_utils import InputExample
from transformers import AdamW
from transformers.optimization import get_linear_schedule_with_warmup
from tqdm import tqdm
import torch

# Setting Dataset 
dataset_path = "/lustre/work/client/users/minhos/cache/datasets/p3_rotten_tomato"
raw_dataset = load_from_disk(dataset_path)
# Load the T5 model
t5_path = "/lustre/work/client/users/minhos/models_for_supercomputer/t5-base"
model = T5ForConditionalGeneration.from_pretrained(t5_path)
tokenizer = T5Tokenizer.from_pretrained(t5_path)



# Prepare dataset to feed dataloader
label_map = {"positive": 0,"negative": 1}

dataset = {}
for split in ['train', 'validation']:
    dataset[split] = []
    raw_dataset[split] = raw_dataset[split].shuffle(seed=42).select(range(1000))
    for idx, data in enumerate(raw_dataset[split]):
        label_text = data["targets_pretokenized"].strip().lower()
        label_numeric = label_map.get(label_text, -1)
        input_example = InputExample(text_a = data['inputs_pretokenized'], guid=idx, label=label_numeric)
        dataset[split].append(input_example)

sampler = FewShotSampler(num_examples_per_label=30)
fewshot_data = sampler(dataset['train'], seed=42)


# Prepare Template, Verbalizer, Model, Dataloader
template = PrefixTuningTemplate(model=model, tokenizer=tokenizer,num_token=50)
verbalizer = ManualVerbalizer(
    tokenizer=tokenizer, 
    num_classes=2,  # Example: binary classification
    label_words=[["positive", "good", "excellent", "wonderful"], ["negative", "bad", "horrible", "awful"]],
    classes=[0, 1]
)

prompt_model = PromptForClassification(plm=model,template=template, verbalizer=verbalizer, freeze_plm=True)

train_dataloader = PromptDataLoader(dataset=fewshot_data, template=template, tokenizer=tokenizer,
    tokenizer_wrapper_class=T5TokenizerWrapper, max_seq_length=480, decoder_max_length=3,
    batch_size=5,shuffle=True, teacher_forcing=False, predict_eos_token=False,
    truncate_method="tail")

validation_dataloader = PromptDataLoader(dataset=dataset["validation"], template=template, tokenizer=tokenizer,
    tokenizer_wrapper_class=T5TokenizerWrapper, max_seq_length=480, decoder_max_length=3,
    batch_size=5,shuffle=False, teacher_forcing=False, predict_eos_token=False,
    truncate_method="tail")




# Define loss function
loss_func = torch.nn.CrossEntropyLoss()

# Define optimizer for the prefix tuning parameters
optimizer_grouped_parameters = [{'params': [p for name, p in template.named_parameters() if 'raw_embedding' not in name]}]
optimizer = AdamW(optimizer_grouped_parameters, lr=1e-4)

# Define a learning rate scheduler
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=20, num_training_steps=1000)



/lustre/work/client/users/minhos/.conda/envs/prompt_learning_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/lustre/work/client/users/minhos/.conda/envs/prompt_learning_env/lib/python3.10/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


{
  "guid": 0,
  "label": 1,
  "meta": {},
  "text_a": ". . . plays like somebody spliced random moments of a chris rock routine into what is otherwise a cliche-riddled but self-serious spy thriller . What is the sentiment expressed by the reviewer for the movie? ",
  "text_b": "",
  "tgt_text": null
}

<class 'openprompt.data_utils.utils.InputExample'>


/lustre/work/client/users/minhos/.conda/envs/prompt_learning_env/lib/python3.10/site-packages/transformers/modeling_utils.py:399: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

In [42]:
from collections import Counter
label_distribution = Counter([example.label for example in fewshot_data])
print("Label Distribution:", label_distribution)
from collections import Counter
label_distribution_val = Counter([example.label for example in dataset["validation"]])
print("Label Distribution:", label_distribution_val)


Label Distribution: Counter({0: 30, 1: 30})
Label Distribution: Counter({1: 501, 0: 499})


In [2]:

# Set the model to training mode
prompt_model.train()

# Define training parameters
num_epochs = 10  # Define the number of epochs
gradient_accumulation_steps = 1  # Define gradient accumulation if needed

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    total_loss = 0
    pbar = tqdm(train_dataloader, desc="Training")
    
    for step, inputs in enumerate(pbar):
        logits = prompt_model(inputs)
        labels = inputs['label']  # Ground-truth labels
        
        # Compute loss
        loss = loss_func(logits, labels)
        loss.backward()  # Backpropagation
        
        # Optimizer step
        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
        
        total_loss += loss.item()
        pbar.set_postfix({"loss": total_loss / (step + 1)})


Epoch 1/10


Training: 100%|██████████| 12/12 [00:20<00:00,  1.74s/it, loss=1.44]


Epoch 2/10


Training: 100%|██████████| 12/12 [00:16<00:00,  1.37s/it, loss=1.16]


Epoch 3/10


...

Training: 100%|██████████| 12/12 [00:14<00:00,  1.18s/it, loss=0.539]


Epoch 9/10


Training: 100%|██████████| 12/12 [00:15<00:00,  1.26s/it, loss=0.495]


Epoch 10/10


Training: 100%|██████████| 12/12 [00:16<00:00,  1.37s/it, loss=0.425]


In [3]:
def evaluate(prompt_model, dataloader):
    prompt_model.eval()  # Set the model to evaluation mode
    total, correct = 0, 0
    
    with torch.no_grad():
        for inputs in dataloader:
            logits = prompt_model(inputs)
            preds = torch.argmax(logits, dim=-1)
            labels = inputs['label']
            
            total += len(labels)
            correct += (preds == labels).sum().item()
    
    accuracy = correct / total
    return accuracy


num_epochs = 1
for epoch in range(num_epochs):
    # Training steps as defined above
    
    # Validation after each epoch
    val_accuracy = evaluate(prompt_model, validation_dataloader)
    print(f"Validation Accuracy after Epoch {epoch + 1}: {val_accuracy:.4f}")


Validation Accuracy after Epoch 1: 0.8480


In [8]:
# This time the verbalizer label: label_words=[["positive", "good", "excellent", "wonderful"], ["negative", "bad", "horrible", "awful"]]
# loss in the beginning: 1.55
# loss after training 10 epochs: 1.26
# Number of soft tokens: 10
# Number of samples per label: 30 
# learning rate=3e-5
# accuracy: 0.5010

# -------------------------- increase the sample from 30 to 100 ---------------------------------------------------------------------

# This time the verbalizer label: label_words=[["positive", "good", "excellent", "wonderful"], ["negative", "bad", "horrible", "awful"]]
# loss in the beginning: 1.55
# loss after training 10 epochs: 0.443
# Number of soft tokens: 10
# Number of samples per label: 100 
# learning rate=3e-5
# accuracy:0.8310

#------------------------------------------lower the number of samples back to 30 ----------------------------------------------------------
#------------------------------------------number of soft token increased from 10 to 100 ---------------------------------------------------

# This time the verbalizer label: label_words=[["positive", "good", "excellent", "wonderful"], ["negative", "bad", "horrible", "awful"]]
# loss in the beginning: 1.51
# loss after training 10 epochs: 1.33
# Number of soft tokens: 100
# Number of samples per label: 30 
# learning rate=3e-5
# accuracy: 0.4990

#-------------------------------------------lowered learning rate from 3e-5 to 1e-4 ---------------------------------------------------

# This time the verbalizer label: label_words=[["positive", "good", "excellent", "wonderful"], ["negative", "bad", "horrible", "awful"]]
# loss in the beginning: 1.57
# loss after training 10 epochs: 0.699
# Number of soft tokens: 100
# Number of samples per label: 30 
# learning rate=1e-4
# accuracy: 0.5360

## so far learning warm up was 500, maybe better to lower it considering the size of the samples (warm-up: 10)

#----------------------------------warmup step change from 500 to 10 --------------------------------------------------

# This time the verbalizer label: label_words=[["positive", "good", "excellent", "wonderful"], ["negative", "bad", "horrible", "awful"]]
# loss in the beginning: 1.41
# loss after training 10 epochs: 0.404
# Number of soft tokens: 100
# Number of samples per label: 30 
# learning rate=1e-4
# accuracy: 0.8470

#----------------------------------------soft token lowered from 100 to 10 -------------------------------------

# loss in the beginning: 1.43
# loss after training 10 epochs: 0.353
# Number of soft tokens: 10
# Number of samples per label: 30 
# learning rate=1e-4
# accuracy: 0.8500

#----------------------------------------learning rate adjustment-----------------------------------------------
# loss in the beginning: 1.43
# loss after training 10 epochs: 0.721
# Number of soft tokens: 10
# Number of samples per label: 30 
# learning rate=0.05
# accuracy: 0.5010


# loss in the beginning: 1.16
# loss after training 10 epochs: 0.75
# Number of soft tokens: 10
# Number of samples per label: 30 
# learning rate=0.01
# accuracy: 0.5010


# loss in the beginning: 1.23
# loss after training 10 epochs: 0.769
# Number of soft tokens: 10
# Number of samples per label: 30 
# learning rate=0.005
# accuracy: 0.4990


# loss in the beginning: 1.14
# loss after training 10 epochs: 0.773
# Number of soft tokens: 10
# Number of samples per label: 30 
# learning rate=0.001
# accuracy: 0.5010


# loss in the beginning: 1.18
# loss after training 10 epochs: 0.253
# Number of soft tokens: 10
# Number of samples per label: 30 
# learning rate=0.0005
# accuracy: 0.8170

#------------------------increase warm up step from 10 to 20 ---------------------------------------------

# loss in the beginning: 1.33
# loss after training 10 epochs: 0.196
# Number of soft tokens: 10
# Number of samples per label: 30 
# learning rate=0.0005
# accuracy: 0.8200

#------------------------increase warm up step from 20 to 25 ---------------------------------------------
# loss in the beginning: 1.38
# loss after training 10 epochs: 0.688
# Number of soft tokens: 10
# Number of samples per label: 30 
# learning rate=0.0005
# accuracy: 0.7850

#------------------------Lower learning rate back to 1e-4 with warm up step 25  ---------------------------------------------
# loss in the beginning: 1.39
# loss after training 10 epochs: 0.494
# Number of soft tokens: 10
# Number of samples per label: 30 
# learning rate=1e-4
# accuracy: 0.8490

#------------------------Lower learning rate back to 1e-4 with warm up step 20  ---------------------------------------------

# loss in the beginning: 1.54
# loss after training 10 epochs: 0.415
# Number of soft tokens: 10
# Number of samples per label: 30 
# learning rate=1e-4
# accuracy: 0.8490

#--------------------Increased the number of soft token from 10 to 100

# loss in the beginning: 1.51
# loss after training 10 epochs: 0.526
# Number of soft tokens: 100
# Number of samples per label: 30 
# learning rate=1e-4
# accuracy: 0.8360

#------------------lowered the number of soft toekn from 100 to 50
# loss in the beginning: 1.44
# loss after training 10 epochs: 0.425
# Number of soft tokens: 50
# Number of samples per label: 30 
# learning rate=1e-4
# accuracy: 0.8480

